<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/all_seeds_neurons_batch50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
mnist_trainset = torchvision.datasets.MNIST(root='./', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root='./', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [4]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self, neuron_size):
        super(Model, self).__init__()
        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, neuron_size)
        self.linear_2 = torch.nn.Linear(neuron_size, 10)
        self.sigmoid12  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        pred = self.linear_2(x)
        return pred
# ************* modify this section for later use *************

In [5]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [6]:
def sparsity_calculator(final_spareness, neuron_size):
    sparseness_list_avg = list()
    sparseness_list_std = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, neuron_size))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)

        mean_sparseness_per_epoch = torch.mean(population_sparseness)
        std_sparseness_per_epoch  = torch.std(population_sparseness)

        sparseness_list_avg.append(mean_sparseness_per_epoch)

    return sparseness_list_avg

In [7]:
def selectivity(hidden_layer_each_neuron, neuron_size):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(neuron_size)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 'neuron_size' lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(neuron_size)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [8]:
# ************* modify this section for later use *************
def avg_std_calculator(_hidden_layer_each_neuron_12, neuron_size):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12, neuron_size)

    final_selectivity_avg = (avg_selectivity12) / 1
    final_selecvitity_std = (std_selectivity12) / 1

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [9]:
def model_factory(optimizer_name, seed_num, neuron_size):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model(neuron_size)
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity_avg, selectivity_list_avg, selectivity_list_std = sparsity_selectivity_trainer(optimizer=my_optimizer, model=my_model, neuron_size=neuron_size)

    # ************* modify this section for later use *************
    file_saver = open(f"seed{seed_num}_batch50_neuronsize{neuron_size}_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity_avg)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    if seed_num == 1:
        if neuron_size == 64:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch50_neuronsize64_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch50_neuronsize64_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch50_neuronsize64_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch50_neuronsize64_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 128:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch50_neuronsize128_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch50_neuronsize128_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch50_neuronsize128_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch50_neuronsize128_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 256:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch50_neuronsize256_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch50_neuronsize256_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch50_neuronsize256_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch50_neuronsize256_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 384:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch50_neuronsize384_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch50_neuronsize384_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch50_neuronsize384_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch50_neuronsize384_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 512:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch50_neuronsize512_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch50_neuronsize512_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch50_neuronsize512_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch50_neuronsize512_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 640:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch50_neuronsize640_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch50_neuronsize640_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch50_neuronsize640_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch50_neuronsize640_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 768:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1_batch50_neuronsize768_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1_batch50_neuronsize768_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1_batch50_neuronsize768_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1_batch50_neuronsize768_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

    elif seed_num == 100:
        if neuron_size == 64:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch50_neuronsize64_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch50_neuronsize64_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch50_neuronsize64_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch50_neuronsize64_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 128:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch50_neuronsize128_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch50_neuronsize128_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch50_neuronsize128_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch50_neuronsize128_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 256:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch50_neuronsize256_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch50_neuronsize256_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch50_neuronsize256_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch50_neuronsize256_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 384:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch50_neuronsize384_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch50_neuronsize384_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch50_neuronsize384_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch50_neuronsize384_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 512:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch50_neuronsize512_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch50_neuronsize512_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch50_neuronsize512_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch50_neuronsize512_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 640:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch50_neuronsize640_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch50_neuronsize640_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch50_neuronsize640_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch50_neuronsize640_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 768:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed100_batch50_neuronsize768_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed100_batch50_neuronsize768_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed100_batch50_neuronsize768_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed100_batch50_neuronsize768_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

    # seed 1234
    elif seed_num == 1234:
        if neuron_size == 64:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch50_neuronsize64_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch50_neuronsize64_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch50_neuronsize64_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch50_neuronsize64_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 128:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch50_neuronsize128_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch50_neuronsize128_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch50_neuronsize128_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch50_neuronsize128_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 256:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch50_neuronsize256_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch50_neuronsize256_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch50_neuronsize256_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch50_neuronsize256_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 384:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch50_neuronsize384_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch50_neuronsize384_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch50_neuronsize384_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch50_neuronsize384_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 512:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch50_neuronsize512_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch50_neuronsize512_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch50_neuronsize512_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch50_neuronsize512_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 640:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch50_neuronsize640_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch50_neuronsize640_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch50_neuronsize640_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch50_neuronsize640_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

        elif neuron_size == 768:
            # ************* modify this section for later use *************
            if optimizer_name == 'Adadelta':
                !cp seed1234_batch50_neuronsize768_Adadelta.txt /content/drive/MyDrive
            
            elif optimizer_name == 'Adagrad':
                !cp seed1234_batch50_neuronsize768_Adagrad.txt /content/drive/MyDrive

            elif optimizer_name == 'SGD':
                !cp seed1234_batch50_neuronsize768_SGD.txt /content/drive/MyDrive

            elif optimizer_name == 'Adam':
                !cp seed1234_batch50_neuronsize768_Adam.txt /content/drive/MyDrive
            # ************* modify this section for later use *************

In [10]:
no_epochs = 30
def sparsity_selectivity_trainer(optimizer, model, neuron_size):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(neuron_size)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************

        total_train_loss = 0
        total_test_loss = 0

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        # ************* modify this section for later use *************

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])

            # ************* modify this section for later use *************
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(neuron_size):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])
    
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12, neuron_size)
        # ************* modify this section for later use *************
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        # ***************** sparsity calculation ***************** #

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    # ***************** sparsity calculation ***************** #
    sparsity_list12_avg = sparsity_calculator(final_spareness_12, neuron_size)

    average_sparsity = list()

    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12_avg[i].item()) / 1 )
    # ***************** sparsity calculation ***************** #

    return test_acc, average_sparsity, selectivity_avg_list, selectivity_std_list

In [11]:
# Seed 1 

torch.manual_seed(1)
np.random.seed(1)

# 64 neurons 
model_factory('Adadelta', 1, 64)
model_factory('Adagrad', 1, 64)
model_factory('SGD', 1, 64)
model_factory('Adam', 1, 64)

# 128 neurons 
model_factory('Adadelta', 1, 128)
model_factory('Adagrad', 1, 128)
model_factory('SGD', 1, 128)
model_factory('Adam', 1, 128)

# 256 neurons 
model_factory('Adadelta', 1, 256)
model_factory('Adagrad', 1, 256)
model_factory('SGD', 1, 256)
model_factory('Adam', 1, 256)

# 384 neurons 
model_factory('Adadelta', 1, 384)
model_factory('Adagrad', 1, 384)
model_factory('SGD', 1, 384)
model_factory('Adam', 1, 384)

# 512 neurons 
model_factory('Adadelta', 1, 512)
model_factory('Adagrad', 1, 512)
model_factory('SGD', 1, 512)
model_factory('Adam', 1, 512)

# 640 neurons 
model_factory('Adadelta', 1, 640)
model_factory('Adagrad', 1, 640)
model_factory('SGD', 1, 640)
model_factory('Adam', 1, 640)

# 768 neurons 
model_factory('Adadelta', 1, 768)
model_factory('Adagrad', 1, 768)
model_factory('SGD', 1, 768)
model_factory('Adam', 1, 768)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 538.11566842, Test Loss: 47.24539231, Test Accuracy: 0.93070000

Epoch: 2/30, Train Loss: 252.60383422, Test Loss: 36.85708197, Test Accuracy: 0.94540000

Epoch: 3/30, Train Loss: 197.72156372, Test Loss: 30.02876011, Test Accuracy: 0.95500000

Epoch: 4/30, Train Loss: 164.63993897, Test Loss: 26.59690505, Test Accuracy: 0.96080000

Epoch: 5/30, Train Loss: 142.09571565, Test Loss: 23.63740453, Test Accuracy: 0.96490000

Epoch: 6/30, Train Loss: 125.21132918, Test Loss: 22.18398830, Test Accuracy: 0.96630000

Epoch: 7/30, Train Loss: 112.72124478, Test Loss: 20.96802351, Test Accuracy: 0.96800000

Epoch: 8/30, Train Loss: 101.58509797, Test Loss: 20.03933046, Test Accuracy: 0.96920000


In [12]:
# Seed 100 

torch.manual_seed(100)
np.random.seed(100)

# 64 neurons 
model_factory('Adadelta', 100, 64)
model_factory('Adagrad', 100, 64)
model_factory('SGD', 100, 64)
model_factory('Adam', 100, 64)

# 128 neurons 
model_factory('Adadelta', 100, 128)
model_factory('Adagrad', 100, 128)
model_factory('SGD', 100, 128)
model_factory('Adam', 100, 128)

# 256 neurons 
model_factory('Adadelta', 100, 256)
model_factory('Adagrad', 100, 256)
model_factory('SGD', 100, 256)
model_factory('Adam', 100, 256)

# 384 neurons 
model_factory('Adadelta', 100, 384)
model_factory('Adagrad', 100, 384)
model_factory('SGD', 100, 384)
model_factory('Adam', 100, 384)

# 512 neurons 
model_factory('Adadelta', 100, 512)
model_factory('Adagrad', 100, 512)
model_factory('SGD', 100, 512)
model_factory('Adam', 100, 512)

# 640 neurons 
model_factory('Adadelta', 100, 640)
model_factory('Adagrad', 100, 640)
model_factory('SGD', 100, 640)
model_factory('Adam', 100, 640)

# 768 neurons 
model_factory('Adadelta', 100, 768)
model_factory('Adagrad', 100, 768)
model_factory('SGD', 100, 768)
model_factory('Adam', 100, 768)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 553.36039481, Test Loss: 47.13900674, Test Accuracy: 0.92800000

Epoch: 2/30, Train Loss: 254.02666945, Test Loss: 36.24725453, Test Accuracy: 0.94580000

Epoch: 3/30, Train Loss: 198.48381351, Test Loss: 30.27196677, Test Accuracy: 0.95550000

Epoch: 4/30, Train Loss: 164.33736990, Test Loss: 25.57852910, Test Accuracy: 0.96130000

Epoch: 5/30, Train Loss: 141.20833545, Test Loss: 23.39238178, Test Accuracy: 0.96450000

Epoch: 6/30, Train Loss: 123.92942991, Test Loss: 21.71510148, Test Accuracy: 0.96740000

Epoch: 7/30, Train Loss: 111.03385659, Test Loss: 20.29218681, Test Accuracy: 0.96910000

Epoch: 8/30, Train Loss: 100.63470343, Test Loss: 19.24119887, Test Accuracy: 0.97070000


In [13]:
# Seed 1234 

torch.manual_seed(1234)
np.random.seed(1234)

# 64 neurons 
model_factory('Adadelta', 1234, 64)
model_factory('Adagrad', 1234, 64)
model_factory('SGD', 1234, 64)
model_factory('Adam', 1234, 64)

# 128 neurons 
model_factory('Adadelta', 1234, 128)
model_factory('Adagrad', 1234, 128)
model_factory('SGD', 1234, 128)
model_factory('Adam', 1234, 128)

# 256 neurons 
model_factory('Adadelta', 1234, 256)
model_factory('Adagrad', 1234, 256)
model_factory('SGD', 1234, 256)
model_factory('Adam', 1234, 256)

# 384 neurons 
model_factory('Adadelta', 1234, 384)
model_factory('Adagrad', 1234, 384)
model_factory('SGD', 1234, 384)
model_factory('Adam', 1234, 384)

# 512 neurons 
model_factory('Adadelta', 1234, 512)
model_factory('Adagrad', 1234, 512)
model_factory('SGD', 1234, 512)
model_factory('Adam', 1234, 512)

# 640 neurons 
model_factory('Adadelta', 1234, 640)
model_factory('Adagrad', 1234, 640)
model_factory('SGD', 1234, 640)
model_factory('Adam', 1234, 640)

# 768 neurons 
model_factory('Adadelta', 1234, 768)
model_factory('Adagrad', 1234, 768)
model_factory('SGD', 1234, 768)
model_factory('Adam', 1234, 768)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/30, Train Loss: 543.26497061, Test Loss: 47.65902656, Test Accuracy: 0.93360000

Epoch: 2/30, Train Loss: 250.98534532, Test Loss: 35.81875902, Test Accuracy: 0.94690000

Epoch: 3/30, Train Loss: 197.05761841, Test Loss: 29.49940958, Test Accuracy: 0.95850000

Epoch: 4/30, Train Loss: 164.58813300, Test Loss: 26.20043305, Test Accuracy: 0.96050000

Epoch: 5/30, Train Loss: 142.23355668, Test Loss: 23.65165100, Test Accuracy: 0.96600000

Epoch: 6/30, Train Loss: 125.98309546, Test Loss: 22.14693882, Test Accuracy: 0.96720000

Epoch: 7/30, Train Loss: 112.46704478, Test Loss: 21.35487642, Test Accuracy: 0.96720000

Epoch: 8/30, Train Loss: 102.19556407, Test Loss: 20.34007097, Test Accuracy: 0.97040000
